In [1]:
import gc
import os
import talib
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone


import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short
from config import bybit_client, prices_collection
from helper import vwap, current_position

load_dotenv()

client = bybit_client()
prices = prices_collection()

In [9]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f'Finding the last record in our DB: \n{last_date}')

print(f'Querying data from bybit to make current')
new_data_query = client.query_kline(symbol='ETHUSD', interval='5', from_time=str(int(datetime.timestamp(last_date))))
new_data_query['result']

Finding the last record in our DB 
2022-12-24 15:25:00


[]

In [2]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f'Finding the last record in our DB')

new_data_query = client.query_kline(symbol='ETHUSD', interval='5', from_time=str(int(datetime.timestamp(last_date))))
print('Getting new data from Bybit API (5m interval)')

new_data = pd.DataFrame(new_data_query['result'], columns=['symbol', 'interval', 'open_time', 'open', 'high', 'low', 'close', 'volume', 'turnover'])
new_data['date'] = pd.to_datetime(new_data['open_time'], unit='s')
new_data.rename(columns={'volume': 'vol'}, inplace=True)
cols_to_drop = ['symbol', 'interval', 'open_time', 'turnover']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print(f'Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'

signal = df['signal'].iloc[-1]
price = df['close'].iloc[-1]
position = current_position()
print(f'Current signal: {signal}, current position: {position}')

Finding the last record in our DB
Getting new data from Bybit API (5m interval)
Calculating signals
Current signal: SELL, current position: SELL


In [3]:
df

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
0,63a5313f3740c2243348e0da,1221.21,1221.61,1221.21,1221.43,2.60083,NaN,NaN,NaN,NaN,NaN,2022-12-23 04:39:59.999,None
1,63a5326c3740c2243348e0dd,1221.85,1222.18,1221.59,1221.87,8.41179,NaN,NaN,NaN,NaN,NaN,2022-12-23 04:44:59.999,None
2,63a533983740c2243348e0e0,1221.69,1222.12,1221.34,1221.61,13.20556,NaN,NaN,NaN,NaN,NaN,2022-12-23 04:49:59.999,None
3,63a534c33740c2243348e0e3,1221.50,1222.99,1221.37,1222.73,26.12496,NaN,NaN,NaN,NaN,NaN,2022-12-23 04:54:59.999,None
4,63a535ef3740c2243348e0e6,1223.18,1223.52,1222.56,1223.28,11.18719,NaN,NaN,NaN,NaN,NaN,2022-12-23 04:59:59.999,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,63a71a6feb2e0eed50509795,1218.05,1218.05,1217.70,1217.75,146534.00000,42.971304,-0.006045,1218.098107,1218.281060,1219.382506,2022-12-24 15:05:00.000,SELL
296,63a71a6feb2e0eed50509796,1217.75,1217.75,1217.70,1217.75,7494.00000,42.971304,-0.025920,1218.028486,1218.262093,1219.366262,2022-12-24 15:10:00.000,SELL
297,63a71a6feb2e0eed50509797,1217.75,1217.85,1217.65,1217.70,9156.00000,42.150175,-0.039945,1217.962789,1218.242019,1219.349683,2022-12-24 15:15:00.000,SELL
298,63a71a6feb2e0eed50509798,1217.70,1217.80,1217.65,1217.80,62352.00000,44.437000,-0.039903,1217.930231,1218.226232,1219.334263,2022-12-24 15:20:00.000,SELL


In [4]:
client.my_position()

{'ret_code': 0,
 'ret_msg': 'OK',
 'result': [{'data': {'id': 0,
    'position_idx': 0,
    'mode': 0,
    'user_id': 48899600,
    'risk_id': 241,
    'symbol': 'ADAUSD',
    'side': 'None',
    'size': 0,
    'position_value': '0',
    'entry_price': '0',
    'is_isolated': False,
    'auto_add_margin': 0,
    'leverage': '10',
    'effective_leverage': '10',
    'position_margin': '0',
    'liq_price': '0',
    'bust_price': '0',
    'occ_closing_fee': '0',
    'occ_funding_fee': '0',
    'take_profit': '0',
    'stop_loss': '0',
    'trailing_stop': '0',
    'position_status': 'Normal',
    'deleverage_indicator': 0,
    'oc_calc_data': '{"blq":0,"slq":0,"bmp":0,"smp":0,"bv2c":0.10126,"sv2c":0.10114}',
    'order_margin': '0',
    'wallet_balance': '0',
    'realised_pnl': '0',
    'unrealised_pnl': 0,
    'cum_realised_pnl': '0',
    'cross_seq': -1,
    'position_seq': 0,
    'created_at': '2022-12-23T15:59:30.475536517Z',
    'updated_at': '2022-12-23T15:59:30.475536517Z',
    '

In [ ]:
signal = df['signal'].iloc[-1]
position = current_position()

if signal == position:
    print('No change in position')
    
else:
    with open('current_position.txt', 'w') as f:
        f.write(signal)

    if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='USD')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='ETH')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'None':
        print('No signal, holding position')
        pass

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))